# Vector Partition Using Namespace

In [ ]:
import subprocess
PINECONE_API_KEY = subprocess.check_output(["direnv", "exec", ".", "bash", "-c", "echo -n ${PINECONE_API_KEY}"], text=True)

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key = PINECONE_API_KEY)

In [ ]:
idx = pc.Index("insert") # connect to index

C:\Users\VENOEN\Pinecone5\pc5_env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
# count of emails
dimensions = 3 # index dimension

emails_with_subject = 20
emails_with_body = 45
emails_with_other = 45

### Upsert

In [6]:
import numpy as np

In [9]:
# create vectors, assume embedded
# python list of lists
vects_subj= np.random.rand(emails_with_subject, dimensions).tolist()
vects_body= np.random.rand(emails_with_body, dimensions).tolist() 
vects_other= np.random.rand(emails_with_other, dimensions).tolist() 

# create ids for these vectors ,IDs are string

ids_subj = map(str, np.arange(emails_with_subject).tolist())
ids_body = map(str, np.arange(emails_with_body).tolist())
ids_other = map(str, np.arange(emails_with_other).tolist())

# zip id and vector
vectors_subj = list(zip(ids_subj, vects_subj)) # list of tuples (id, values)
vectors_body = list(zip(ids_body, vects_body) )
vectors_other = list(zip(ids_other, vects_other))


# upsert vectors into index with different namespace

idx.upsert(vectors_subj, namespace='subject')
idx.upsert(vectors_body, namespace='body')
idx.upsert(vectors_other) # default namespace is ""

{'upserted_count': 45}

### Query

In [10]:
idx.query(vector = list(np.random.rand(3)), 
          top_k=3, 
          namespace='', 
          include_values=True)

{'matches': [{'id': '26',
              'score': 0.999655485,
              'values': [0.893857181, 0.490591437, 0.930496931]},
             {'id': '3',
              'score': 0.999513686,
              'values': [0.88356185, 0.514331579, 0.95760417]},
             {'id': '20',
              'score': 0.998717487,
              'values': [0.788190603, 0.490408182, 0.880566716]}],
 'namespace': '',
 'usage': {'read_units': 6}}

### Update and Fetch

In [11]:
vectors_other[44]  # current value

('44', [0.37197518045529443, 0.30628789659583133, 0.9388183480600835])

In [12]:
new_value = [0.1, 0.1, 0.1]

In [13]:
idx.update(id = '44', 
          values= new_value,
          namespace = '')

{}

In [14]:
idx.fetch(ids = ['44'], namespace = '')

{'namespace': '',
 'usage': {'read_units': 1},
 'vectors': {'44': {'id': '44', 'values': [0.1, 0.1, 0.1]}}}

### Delete

In [15]:
idx.delete(ids = ['0', '1'], namespace='subject')

{}

In [16]:
idx.delete(ids = ['0', '1', '2'], namespace='body')

{}

In [17]:
idx.delete(delete_all=True) # deleted vector without namespace

{}

In [18]:
idx.delete(delete_all=True, namespace='body')

{}